## Import Libraries

In [150]:
import pandas as pd
import numpy as np
import smtplib, ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from sqlalchemy import create_engine
from config import dbuser, dbpwd, dbhost, surveyId

## Read New CSV into DF

In [125]:
path = 'MyQualtricsDownload/B2BSurvey.csv'

In [126]:
df = pd.read_csv(path, encoding = "ISO-8859-1")
df

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,...,Topic_1_Q3,Topic_2_Q1,Topic_2_Q2,Topic_2_Q3,Topic_3_Q1,Topic_3_Q2,Topic_3_Q3,Referal_4,Referal_5,Referal_6
0,Start Date,End Date,Response Type,IP Address,Progress,Duration (in seconds),Finished,Recorded Date,Response ID,Recipient Last Name,...,Click to write the question text,Click to write the question text,Click to write the question text,Click to write the question text,Click to write the question text,Click to write the question text,Click to write the question text,If you have a colleague that could answer the ...,If you have a colleague that could answer the ...,If you have a colleague that could answer the ...
1,"{""ImportId"":""startDate"",""timeZone"":""Z""}","{""ImportId"":""endDate"",""timeZone"":""Z""}","{""ImportId"":""status""}","{""ImportId"":""ipAddress""}","{""ImportId"":""progress""}","{""ImportId"":""duration""}","{""ImportId"":""finished""}","{""ImportId"":""recordedDate"",""timeZone"":""Z""}","{""ImportId"":""_recordId""}","{""ImportId"":""recipientLastName""}",...,"{""ImportId"":""QID3""}","{""ImportId"":""QID5""}","{""ImportId"":""QID6""}","{""ImportId"":""QID7""}","{""ImportId"":""QID8""}","{""ImportId"":""QID9""}","{""ImportId"":""QID10""}","{""ImportId"":""QID4_4""}","{""ImportId"":""QID4_5""}","{""ImportId"":""QID4_6""}"
2,2020-05-13 13:21:09,2020-05-13 13:21:49,0,74.96.76.163,100,39,1,2020-05-13 13:21:49,R_3hbTUPHEKabGVqN,NaN,...,1,2,1,1,NaN,NaN,NaN,King Boo,Ghost,kingboo32@mailinator.com


In [127]:
#Drop the first two rows
df = df.drop([0,1])
df = df.iloc[:, np.r_[7,17:36]]
df.head()

,RecordedDate,Contact_Info_1,Contact_Info_2,Contact_Info_3,Contact_Info_8,Topic_1,Topic_2,Topic_3,Topic_1_Q1,Topic_1_Q2,Topic_1_Q3,Topic_2_Q1,Topic_2_Q2,Topic_2_Q3,Topic_3_Q1,Topic_3_Q2,Topic_3_Q3,Referal_4,Referal_5,Referal_6
2,2020-05-13 13:21:49,Monty Mole,Mole,monty_mole@mailinator.com,Nintendo,1,1,NaN,2,3,1,2,1,1,NaN,NaN,NaN,King Boo,Ghost,kingboo32@mailinator.com


In [128]:
#Rename contact and referal info
df = df.rename({'Contact_Info_1': 'Contact_Name', 'Contact_Info_2': 'Contact_Title',
                'Contact_Info_3':'Contact_Email','Contact_Info_8':'Contact_Company',
                'Referal_4':'Referal_Name','Referal_5':'Referal_Title','Referal_6':'Referal_Email'}, axis=1)
df.head()

,RecordedDate,Contact_Name,Contact_Title,Contact_Email,Contact_Company,Topic_1,Topic_2,Topic_3,Topic_1_Q1,Topic_1_Q2,Topic_1_Q3,Topic_2_Q1,Topic_2_Q2,Topic_2_Q3,Topic_3_Q1,Topic_3_Q2,Topic_3_Q3,Referal_Name,Referal_Title,Referal_Email
2,2020-05-13 13:21:49,Monty Mole,Mole,monty_mole@mailinator.com,Nintendo,1,1,NaN,2,3,1,2,1,1,NaN,NaN,NaN,King Boo,Ghost,kingboo32@mailinator.com


## Prepare Data to Update SQL Tables

In [129]:
#Cloud Postgresql Database Connection on AWS
engine = create_engine(f"postgresql://{dbuser}:{dbpwd}@{dbhost}:{dbport}/{dbname}")

In [130]:
#Create a remote database engine connection
conn = engine.connect()

### Results

#### Topic 1

In [131]:
#Subset df based on who has responded to first question in topic 1
T1 = df[df['Topic_1_Q1'].notnull()]
T1.head()

,RecordedDate,Contact_Name,Contact_Title,Contact_Email,Contact_Company,Topic_1,Topic_2,Topic_3,Topic_1_Q1,Topic_1_Q2,Topic_1_Q3,Topic_2_Q1,Topic_2_Q2,Topic_2_Q3,Topic_3_Q1,Topic_3_Q2,Topic_3_Q3,Referal_Name,Referal_Title,Referal_Email
2,2020-05-13 13:21:49,Monty Mole,Mole,monty_mole@mailinator.com,Nintendo,1,1,NaN,2,3,1,2,1,1,NaN,NaN,NaN,King Boo,Ghost,kingboo32@mailinator.com


In [132]:
#Select columns to keep by index
T1 = T1.iloc[:,np.r_[0:5,8:11]]
T1.head()

,RecordedDate,Contact_Name,Contact_Title,Contact_Email,Contact_Company,Topic_1_Q1,Topic_1_Q2,Topic_1_Q3
2,2020-05-13 13:21:49,Monty Mole,Mole,monty_mole@mailinator.com,Nintendo,2,3,1


In [20]:
T1.to_sql(name='topic_1', if_exists='replace', con=conn, chunksize=25, index=False)

### Update Contacts

In [133]:
#Preate fake contact list
Dict = {'Name': 'Monty Mole','Title':'Mole','Email':'monty_mole@mailinator.com',
        'Company':'Nintendo','Referer':'','CompletedDate':'',
        'T1_Completed':[0],'T2_Completed':[0],'T3_Completed':[0]}
Contacts = pd.DataFrame(Dict)
Contacts.head()

,ContactID,Name,Title,Email,Company,Referer,CompletedDate,T1_Completed,T2_Completed,T3_Completed
0,1,Monty Mole,Mole,monty_mole@mailinator.com,Nintendo,,,0,0,0


In [57]:
Contacts.to_sql(name='contacts', if_exists='replace', con=conn, chunksize=25, index=False)

In [134]:
#Combine existing contacts list with simplified responses
df = pd.merge(Contacts,df,left_on='Email',right_on='Contact_Email')
df.head()

,ContactID,Name,Title,Email,Company,Referer,CompletedDate,T1_Completed,T2_Completed,T3_Completed,...,Topic_1_Q3,Topic_2_Q1,Topic_2_Q2,Topic_2_Q3,Topic_3_Q1,Topic_3_Q2,Topic_3_Q3,Referal_Name,Referal_Title,Referal_Email
0,1,Monty Mole,Mole,monty_mole@mailinator.com,Nintendo,,,0,0,0,...,1,2,1,1,NaN,NaN,NaN,King Boo,Ghost,kingboo32@mailinator.com


In [135]:
#If first topic question is not blank, mark that topic as completed
df.loc[df['Topic_1_Q1'].notnull(), 'T1_Completed'] = 1
df.loc[df['Topic_2_Q1'].notnull(), 'T2_Completed'] = 1
df.loc[df['Topic_3_Q1'].notnull(), 'T3_Completed'] = 1
#If blank, make 0
df.loc[~df['Topic_1_Q1'].notnull(), 'T1_Completed'] = 0
df.loc[~df['Topic_2_Q1'].notnull(), 'T2_Completed'] = 0
df.loc[~df['Topic_3_Q1'].notnull(), 'T3_Completed'] = 0
df.head()

,ContactID,Name,Title,Email,Company,Referer,CompletedDate,T1_Completed,T2_Completed,T3_Completed,...,Topic_1_Q3,Topic_2_Q1,Topic_2_Q2,Topic_2_Q3,Topic_3_Q1,Topic_3_Q2,Topic_3_Q3,Referal_Name,Referal_Title,Referal_Email
0,1,Monty Mole,Mole,monty_mole@mailinator.com,Nintendo,,,1,1,0,...,1,2,1,1,NaN,NaN,NaN,King Boo,Ghost,kingboo32@mailinator.com


In [136]:
#Convert recorded date to 'YYYY-MM-DD' format by extracting before space
df['CompletedDate'] = df['RecordedDate'].str.split(' ').str[0]
df.head()

,ContactID,Name,Title,Email,Company,Referer,CompletedDate,T1_Completed,T2_Completed,T3_Completed,...,Topic_1_Q3,Topic_2_Q1,Topic_2_Q2,Topic_2_Q3,Topic_3_Q1,Topic_3_Q2,Topic_3_Q3,Referal_Name,Referal_Title,Referal_Email
0,1,Monty Mole,Mole,monty_mole@mailinator.com,Nintendo,,2020-05-13,1,1,0,...,1,2,1,1,NaN,NaN,NaN,King Boo,Ghost,kingboo32@mailinator.com


In [143]:
#Remove extra columns from merge
Contacts = df.iloc[:,0:10]
Contacts.head()

,ContactID,Name,Title,Email,Company,Referer,CompletedDate,T1_Completed,T2_Completed,T3_Completed
0,1,Monty Mole,Mole,monty_mole@mailinator.com,Nintendo,,2020-05-13,1,1,0


### New Contacts from Referals

In [144]:
df.head()

,ContactID,Name,Title,Email,Company,Referer,CompletedDate,T1_Completed,T2_Completed,T3_Completed,...,Topic_1_Q3,Topic_2_Q1,Topic_2_Q2,Topic_2_Q3,Topic_3_Q1,Topic_3_Q2,Topic_3_Q3,Referal_Name,Referal_Title,Referal_Email
0,1,Monty Mole,Mole,monty_mole@mailinator.com,Nintendo,,2020-05-13,1,1,0,...,1,2,1,1,NaN,NaN,NaN,King Boo,Ghost,kingboo32@mailinator.com


In [145]:
NewContacts = df[['Referal_Name','Referal_Title','Referal_Email','Company','Name','T1_Completed','T2_Completed','T3_Completed']]
NewContacts.head()

,Referal_Name,Referal_Title,Referal_Email,Company,Name,T1_Completed,T2_Completed,T3_Completed
0,King Boo,Ghost,kingboo32@mailinator.com,Nintendo,Monty Mole,1,1,0


In [146]:
NewContacts = NewContacts.rename({'Referal_Name': 'Name', 'Referal_Title': 'Title',
                                  'Referal_Email':'Email','Name':'Referer'}, axis=1)
NewContacts

,Name,Title,Email,Company,Referer,T1_Completed,T2_Completed,T3_Completed
0,King Boo,Ghost,kingboo32@mailinator.com,Nintendo,Monty Mole,1,1,0


In [147]:
#What if the same person gets recommended? merge rows
aggregation_functions = {'Company':'first', 'Name':'first', 'Email':'first', 'Referer':'first',
                         'T1_Completed':'sum', 'T2_Completed':'sum', 'T3_Completed':'sum'}
NewContacts = NewContacts.groupby(NewContacts['Email']).aggregate(aggregation_functions)

del NewContacts['Email']
NewContacts = NewContacts.reset_index()

In [148]:
#if completed values are not 0, make them 1
NewContacts.loc[NewContacts['T1_Completed'] > 0, 'T1_Completed'] = 1
NewContacts.loc[NewContacts['T2_Completed'] > 0, 'T2_Completed'] = 1
NewContacts.loc[NewContacts['T3_Completed'] > 0, 'T3_Completed'] = 1
NewContacts

,Email,Company,Name,Referer,T1_Completed,T2_Completed,T3_Completed
0,kingboo32@mailinator.com,Nintendo,King Boo,Monty Mole,1,1,0


In [ ]:
#Combine contacts with new contacts
Contacts = pd.concat([Contacts, NewContacts], ignore_index=True)
#Push to db
Contacts.to_sql(name='contacts', if_exists='replace', con=conn, chunksize=25, index=False)